In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
# Reading the data
df = pd.read_csv("Resources/NDXT0919.csv")
print(df.shape)
print(df.info())
df.head(5)

(3055, 8)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3055 entries, 0 to 3054
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       2768 non-null   object 
 1   Open       2767 non-null   float64
 2   High       2767 non-null   float64
 3   Low        2767 non-null   float64
 4   Close      2767 non-null   float64
 5   Adj Close  2767 non-null   float64
 6   Volume     2767 non-null   float64
 7   Class      2768 non-null   object 
dtypes: float64(6), object(2)
memory usage: 191.1+ KB
None


,Date,Open,High,Low,Close,Adj Close,Volume,Class
0,1/2/2009,633.739990,665.849976,631.969971,664.190002,664.190002,0.0,Train
1,1/5/2009,658.140015,672.940002,654.599976,665.950012,665.950012,0.0,Train
2,1/6/2009,671.210022,694.500000,670.080017,687.130005,687.130005,0.0,Train
3,1/7/2009,670.919983,673.989990,657.270020,662.739990,662.739990,0.0,Train
4,1/8/2009,657.210022,665.369995,648.539978,665.099976,665.099976,0.0,Train


In [2]:
df.dropna(inplace=True)

In [3]:
import datetime
df['Date'] = pd.to_datetime(df['Date'])
df['Date'] = df['Date'].dt.strftime('%d.%m.%Y')
df['year'] = pd.DatetimeIndex(df['Date']).year
df['month'] = pd.DatetimeIndex(df['Date']).month
df['day'] = pd.DatetimeIndex(df['Date']).day
df['dayofyear'] = pd.DatetimeIndex(df['Date']).dayofyear
df['weekofyear'] = pd.DatetimeIndex(df['Date']).weekofyear
df['weekday'] = pd.DatetimeIndex(df['Date']).weekday
df['quarter'] = pd.DatetimeIndex(df['Date']).quarter
df['is_month_start'] = pd.DatetimeIndex(df['Date']).is_month_start
df['is_month_end'] = pd.DatetimeIndex(df['Date']).is_month_end
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2767 entries, 0 to 2767
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Date            2767 non-null   object 
 1   Open            2767 non-null   float64
 2   High            2767 non-null   float64
 3   Low             2767 non-null   float64
 4   Close           2767 non-null   float64
 5   Adj Close       2767 non-null   float64
 6   Volume          2767 non-null   float64
 7   Class           2767 non-null   object 
 8   year            2767 non-null   int64  
 9   month           2767 non-null   int64  
 10  day             2767 non-null   int64  
 11  dayofyear       2767 non-null   int64  
 12  weekofyear      2767 non-null   int64  
 13  weekday         2767 non-null   int64  
 14  quarter         2767 non-null   int64  
 15  is_month_start  2767 non-null   bool   
 16  is_month_end    2767 non-null   bool   
dtypes: bool(2), float64(6), int64(7),

C:\Users\rvara\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  


In [4]:
df = df.drop(['Date'], axis = 1) 

In [5]:
df = pd.get_dummies(df, columns=['year'], drop_first=True, prefix='year')

df = pd.get_dummies(df, columns=['month'], drop_first=True, prefix='month')

df = pd.get_dummies(df, columns=['weekday'], drop_first=True, prefix='wday')
df = pd.get_dummies(df, columns=['quarter'], drop_first=True, prefix='qrtr')

df = pd.get_dummies(df, columns=['is_month_start'], drop_first=True, prefix='m_start')

df = pd.get_dummies(df, columns=['is_month_end'], drop_first=True, prefix='m_end')

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2767 entries, 0 to 2767
Data columns (total 42 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Open          2767 non-null   float64
 1   High          2767 non-null   float64
 2   Low           2767 non-null   float64
 3   Close         2767 non-null   float64
 4   Adj Close     2767 non-null   float64
 5   Volume        2767 non-null   float64
 6   Class         2767 non-null   object 
 7   day           2767 non-null   int64  
 8   dayofyear     2767 non-null   int64  
 9   weekofyear    2767 non-null   int64  
 10  year_2010     2767 non-null   uint8  
 11  year_2011     2767 non-null   uint8  
 12  year_2012     2767 non-null   uint8  
 13  year_2013     2767 non-null   uint8  
 14  year_2014     2767 non-null   uint8  
 15  year_2015     2767 non-null   uint8  
 16  year_2016     2767 non-null   uint8  
 17  year_2017     2767 non-null   uint8  
 18  year_2018     2767 non-null 

In [6]:
train = df[df["Class"] == "Train"] 
test = df[df["Class"] == "Test"] 

print(train.shape)
print(test.shape)

(2515, 42)
(252, 42)


In [7]:
train = train.drop(['Class'], axis = 1) 
test = test.drop(['Class'], axis = 1) 

In [8]:
target_column_train = ['Close'] 
predictors_train = list(set(list(train.columns))-set(target_column_train))

X_train = train[predictors_train].values
y_train = train[target_column_train].values

print(X_train.shape)
print(y_train.shape)

(2515, 40)
(2515, 1)


In [9]:
target_column_test = ['Close'] 
predictors_test = list(set(list(test.columns))-set(target_column_test))

X_test = test[predictors_test].values
y_test = test[target_column_test].values

print(X_test.shape)
print(y_test.shape)

(252, 40)
(252, 1)


In [10]:
#depending on the env. pip install sklearn

In [11]:
# CART Model: Classification and regression tree model

In [12]:
import sklearn
from sklearn import model_selection
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from math import sqrt

In [13]:
dtree = DecisionTreeRegressor(max_depth=20, min_samples_leaf=1, random_state=10)
dtree.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=20, random_state=10)

In [14]:
# Code lines 1 to 3
pred_train_tree= dtree.predict(X_train)
print(np.sqrt(mean_squared_error(y_train,pred_train_tree)))
print(r2_score(y_train, pred_train_tree))

# Code lines 4 to 6
pred_test_tree= dtree.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,pred_test_tree))) 
print(r2_score(y_test, pred_test_tree))

0.0
1.0
379.76053340379735
-0.02866696457477369


In [15]:
dtree1 = DecisionTreeRegressor(max_depth=2)
dtree2 = DecisionTreeRegressor(max_depth=8)
dtree3 = DecisionTreeRegressor(max_depth=100)
dtree4 = DecisionTreeRegressor(max_depth=25)
dtree1.fit(X_train, y_train)
dtree2.fit(X_train, y_train)
dtree3.fit(X_train, y_train)
dtree4.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=25)

In [16]:
# # Code Lines 1 to 4: Fit the regression tree 'dtree1' and 'dtree2' 
# dtree1 = DecisionTreeRegressor(max_depth=2)
# dtree2 = DecisionTreeRegressor(max_depth=5)
# dtree1.fit(X_train, y_train)
# dtree2.fit(X_train, y_train)

# # Code Lines 5 to 6: Predict on training data
# tr1 = dtree1.predict(X_train)
# tr2 = dtree2.predict(X_train) 

# #Code Lines 7 to 8: Predict on testing data
# y1 = dtree1.predict(X_test)
# y2 = dtree2.predict(X_test)

In [17]:
tr1 = dtree1.predict(X_train)
tr2 = dtree2.predict(X_train)
tr3 = dtree3.predict(X_train)
tr4 = dtree3.predict(X_train)

y1 = dtree1.predict(X_test)
y2 = dtree2.predict(X_test)
y3 = dtree3.predict(X_test)
y4 = dtree4.predict(X_test)

In [18]:
print(np.sqrt(mean_squared_error(y_train,tr1))) 
print(r2_score(y_train, tr1))

# Print RMSE and R-squared value for regression tree 'dtree1' on testing data
print(np.sqrt(mean_squared_error(y_test,y1))) 
print(r2_score(y_test, y1))

244.32692363344987
0.942369454683905
666.4455179644643
-2.167994642462714


In [19]:
# Print RMSE and R-squared value for regression tree 'dtree2' on training data
print(np.sqrt(mean_squared_error(y_train,tr2))) 
print(r2_score(y_train, tr2))

# Print RMSE and R-squared value for regression tree 'dtree2' on testing data
print(np.sqrt(mean_squared_error(y_test,y2))) 
print(r2_score(y_test, y2))


4.1243629300828495
0.9999835780840493
382.05894096597723
-0.041156153853834976


In [20]:
# Print RMSE and R-squared value for regression tree 'dtree3' on training data
print(np.sqrt(mean_squared_error(y_train,tr3))) 
print(r2_score(y_train, tr3))

# Print RMSE and R-squared value for regression tree 'dtree3' on testing data
print(np.sqrt(mean_squared_error(y_test,y3))) 
print(r2_score(y_test, y3))

0.0
1.0
380.3895030880824
-0.03207719849182644


In [21]:
# Print RMSE and R-squared value for regression tree 'dtree4' on training data
print(np.sqrt(mean_squared_error(y_train,tr4))) 
print(r2_score(y_train, tr4))

# Print RMSE and R-squared value for regression tree 'dtree4' on testing data
print(np.sqrt(mean_squared_error(y_test,y4))) 
print(r2_score(y_test, y4))

0.0
1.0
379.1215070533275
-0.025207983568967363


In [22]:
#RF model
model_rf = RandomForestRegressor(n_estimators=5000, oob_score=True, random_state=100)
model_rf.fit(X_train, y_train) 
pred_train_rf= model_rf.predict(X_train)
print(np.sqrt(mean_squared_error(y_train,pred_train_rf)))
print(r2_score(y_train, pred_train_rf))

pred_test_rf = model_rf.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,pred_test_rf)))
print(r2_score(y_test, pred_test_rf))

C:\Users\rvara\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


1.1323711110934715
0.9999987620940857
383.47110950708196
-0.048867033262709025
